In [2]:
import duckdb
import pandas as pd

In [2]:
# TODO: explain why we create a connection
con = duckdb.connect()
con.sql("SELECT 42 as X").show()
con.close()

┌───────┐
│   X   │
│ int32 │
├───────┤
│    42 │
└───────┘



In [3]:
with duckdb.connect("duckdbdata/file.db") as con:
    con.sql("CREATE TABLE test (i INTEGER)")
    con.sql("INSERT INTO test VALUES (42)")
    con.table("test").show()

┌───────┐
│   i   │
│ int32 │
├───────┤
│    42 │
└───────┘



In [42]:
con = duckdb.connect("../duckdbdata/chicago.db")

In [5]:
df1 = pd.read_parquet("../duckdbdata/ChicagoParkingTickets.parquet")

In [ ]:
con.execute("CREATE TABLE ChicagoParkingTickets AS SELECT * FROM df1")

In [6]:
con.table("ChicagoParkingTickets").show()

┌───────────────┬─────────────────────┬────────────────┬────────────────────┬────────────────────┬────────────────┬───────────────────┬────────────────────┬─────────────────────────┬──────────────────────────────────┬──────────────┬───────┬──────────┬───────┬─────────────────┬────────────┬─────────────────────┬─────────┬──────────────┬────────────┬──────────────────────┐
│ Ticket_number │     Issued_date     │ Community_Name │       Sector       │        Side        │ Hardship_Index │ Per_capita_income │ Percent_unemployed │ Percent_without_diploma │ Percent_households_below_poverty │ Neighborhood │ Ward  │  Tract   │  ZIP  │ Police_District │ Plate_Type │ License_Plate_State │ Unit_ID │ Violation_ID │ Officer_ID │ PaymentIsOutstanding │
│     int64     │       varchar       │    varchar     │      varchar       │      varchar       │     double     │      double       │       double       │         double          │              double              │   varchar    │ int64 │  double  │ 

In [6]:
con.sql("DESCRIBE ChicagoParkingTickets")

┌──────────────────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│           column_name            │ column_type │  null   │   key   │ default │  extra  │
│             varchar              │   varchar   │ varchar │ varchar │ varchar │ varchar │
├──────────────────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ Ticket_number                    │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ Issued_date                      │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Community_Name                   │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Sector                           │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Side                             │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Hardship_Index                   │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ Per_capita_income                │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │

In [7]:
con.sql("SUMMARIZE ChicagoParkingTickets")
# con.sql("CREATE TABLE cpt_summary AS SELECT * FROM (SUMMARIZE ChicagoParkingTickets)")

┌──────────────────────────────────┬─────────────┬─────────────────────────┬─────────────────────┬───────────────┬────────────────────┬────────────────────┬────────────────────┬────────────────────┬────────────────────┬──────────┬─────────────────┐
│           column_name            │ column_type │           min           │         max         │ approx_unique │        avg         │        std         │        q25         │        q50         │        q75         │  count   │ null_percentage │
│             varchar              │   varchar   │         varchar         │       varchar       │     int64     │      varchar       │      varchar       │      varchar       │      varchar       │      varchar       │  int64   │  decimal(9,2)   │
├──────────────────────────────────┼─────────────┼─────────────────────────┼─────────────────────┼───────────────┼────────────────────┼────────────────────┼────────────────────┼────────────────────┼────────────────────┼──────────┼─────────────────┤
│ Ti

In [8]:
con.sql("SELECT COUNT(*) FROM ChicagoParkingTickets")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│     49950127 │
└──────────────┘

In [7]:
con.sql("SELECT * FROM cpt_summary")

┌──────────────────────────────────┬─────────────┬─────────────────────────┬─────────────────────┬───────────────┬────────────────────┬─────────────────────┬────────────────────┬────────────────────┬────────────────────┬──────────┬─────────────────┐
│           column_name            │ column_type │           min           │         max         │ approx_unique │        avg         │         std         │        q25         │        q50         │        q75         │  count   │ null_percentage │
│             varchar              │   varchar   │         varchar         │       varchar       │     int64     │      varchar       │       varchar       │      varchar       │      varchar       │      varchar       │  int64   │  decimal(9,2)   │
├──────────────────────────────────┼─────────────┼─────────────────────────┼─────────────────────┼───────────────┼────────────────────┼─────────────────────┼────────────────────┼────────────────────┼────────────────────┼──────────┼─────────────────┤


In [17]:
con.sql("""
    SELECT
        Police_District,
        CASE WHEN License_Plate_State = 'IL' THEN 'In-State' ELSE 'Out-of-State' END AS License_Plate_State,
        SUM(PaymentIsOutstanding) AS TicketsOutstanding
    FROM ChicagoParkingTickets
    GROUP BY
            Police_District,
            CASE WHEN License_Plate_State = 'IL' THEN 'In-State' ELSE 'Out-of-State' END
    ORDER BY SUM(PaymentIsOutstanding) DESC
""")

┌─────────────────┬─────────────────────┬────────────────────┐
│ Police_District │ License_Plate_State │ TicketsOutstanding │
│     double      │       varchar       │       int128       │
├─────────────────┼─────────────────────┼────────────────────┤
│             1.0 │ In-State            │            1073606 │
│            10.0 │ In-State            │            1018027 │
│            12.0 │ In-State            │             863646 │
│            18.0 │ In-State            │             836144 │
│             9.0 │ In-State            │             730582 │
│             8.0 │ In-State            │             717649 │
│            19.0 │ In-State            │             702865 │
│            25.0 │ In-State            │             535055 │
│            14.0 │ In-State            │             520815 │
│            15.0 │ In-State            │             509890 │
│              ·  │    ·                │                ·   │
│              ·  │    ·                │              

In [30]:
con.sql("""
    WITH records AS (
    SELECT
        Police_District,
        CASE WHEN License_Plate_State = 'IL' THEN 'In-State' ELSE 'Out-of-State' END AS License_Plate_State,
        SUM(PaymentIsOutstanding) AS TicketsOutstanding
    FROM ChicagoParkingTickets
    GROUP BY
            Police_District,
            CASE WHEN License_Plate_State = 'IL' THEN 'In-State' ELSE 'Out-of-State' END
    )
    PIVOT records
    ON License_Plate_State
    USING SUM(TicketsOutstanding)
    ORDER BY Police_District
""")

┌─────────────────┬──────────┬──────────────┐
│ Police_District │ In-State │ Out-of-State │
│     double      │  int128  │    int128    │
├─────────────────┼──────────┼──────────────┤
│             1.0 │  1073606 │       227079 │
│             2.0 │   477357 │        72515 │
│             3.0 │   418782 │        40902 │
│             4.0 │   388914 │        37860 │
│             5.0 │   250010 │        18587 │
│             6.0 │   427142 │        35541 │
│             7.0 │   432358 │        25163 │
│             8.0 │   717649 │        67161 │
│             9.0 │   730582 │        82071 │
│            10.0 │  1018027 │        78570 │
│            11.0 │   482732 │        26311 │
│            12.0 │   863646 │       148139 │
│            14.0 │   520815 │        85161 │
│            15.0 │   509890 │        27189 │
│            16.0 │   244237 │        19435 │
│            17.0 │   393459 │        39429 │
│            18.0 │   836144 │       280370 │
│            19.0 │   702865 │    

In [29]:
con.sql("""
    WITH records AS (
    SELECT
        Police_District,
        CASE WHEN License_Plate_State = 'IL' THEN 'In-State' ELSE 'Out-of-State' END AS License_Plate_State,
        SUM(PaymentIsOutstanding) AS TicketsOutstanding
    FROM ChicagoParkingTickets
    GROUP BY
            Police_District,
            CASE WHEN License_Plate_State = 'IL' THEN 'In-State' ELSE 'Out-of-State' END
    )
    PIVOT records
    ON Police_District IN (1.0, 2.0, 3.0, 4.0)
    USING SUM(TicketsOutstanding)
""")

┌─────────────────────┬─────────┬────────┬────────┬────────┐
│ License_Plate_State │   1.0   │  2.0   │  3.0   │  4.0   │
│       varchar       │ int128  │ int128 │ int128 │ int128 │
├─────────────────────┼─────────┼────────┼────────┼────────┤
│ Out-of-State        │  227079 │  72515 │  40902 │  37860 │
│ In-State            │ 1073606 │ 477357 │ 418782 │ 388914 │
└─────────────────────┴─────────┴────────┴────────┴────────┘

In [40]:
con.sql("""    
    WITH records AS (
        SELECT
            Police_District,
            License_Plate_State,
            PaymentIsOutstanding
        FROM ChicagoParkingTickets
    )
    PIVOT records
    ON Police_District IN (1.0, 2.0, 3.0, 4.0)
    USING SUM(PaymentIsOutstanding)
    ORDER BY License_Plate_State
""").show(max_rows=100)

┌─────────────────────┬─────────┬────────┬────────┬────────┐
│ License_Plate_State │   1.0   │  2.0   │  3.0   │  4.0   │
│       varchar       │ int128  │ int128 │ int128 │ int128 │
├─────────────────────┼─────────┼────────┼────────┼────────┤
│ AB                  │     156 │     79 │     27 │     13 │
│ AK                  │     317 │    128 │     74 │     35 │
│ AL                  │    1634 │    706 │    513 │    366 │
│ AR                  │    1197 │    447 │    335 │    181 │
│ AZ                  │    3248 │    974 │    621 │    341 │
│ BC                  │     260 │     38 │      3 │      3 │
│ CA                  │    9258 │   4620 │   1041 │    605 │
│ CO                  │    2340 │    731 │    216 │     80 │
│ CT                  │    1066 │    409 │     67 │     42 │
│ DC                  │     192 │    135 │     17 │     11 │
│ DE                  │     185 │    123 │     26 │     10 │
│ FL                  │    6504 │   1671 │    680 │    352 │
│ GA                  │ 

In [43]:
con.sql("""    
    WITH records AS (
        SELECT
            Police_District,
            Officer_ID,
            License_Plate_State,
            PaymentIsOutstanding
        FROM ChicagoParkingTickets
    )
    PIVOT records
    ON Police_District IN (1.0, 2.0, 3.0, 4.0)
    USING SUM(PaymentIsOutstanding), MAX(Officer_ID)
    ORDER BY License_Plate_State
""").show(max_rows=100)

┌─────────────────────┬───────────────────────────────┬─────────────────────┬───────────────────────────────┬─────────────────────┬───────────────────────────────┬─────────────────────┬───────────────────────────────┬─────────────────────┐
│ License_Plate_State │ 1.0_sum(PaymentIsOutstanding) │ 1.0_max(Officer_ID) │ 2.0_sum(PaymentIsOutstanding) │ 2.0_max(Officer_ID) │ 3.0_sum(PaymentIsOutstanding) │ 3.0_max(Officer_ID) │ 4.0_sum(PaymentIsOutstanding) │ 4.0_max(Officer_ID) │
│       varchar       │            int128             │        int64        │            int128             │        int64        │            int128             │        int64        │            int128             │        int64        │
├─────────────────────┼───────────────────────────────┼─────────────────────┼───────────────────────────────┼─────────────────────┼───────────────────────────────┼─────────────────────┼───────────────────────────────┼─────────────────────┤
│ AB                  │                 

In [50]:
con.sql("""    
    WITH records AS (
        SELECT
            Police_District,
            License_Plate_State,
            PaymentIsOutstanding
        FROM ChicagoParkingTickets
    ),
    pivoted AS (
        PIVOT records
        ON Police_District IN (1.0, 2.0, 3.0, 4.0)
        USING SUM(PaymentIsOutstanding)
    )
    UNPIVOT pivoted
    ON COLUMNS(* EXCLUDE(License_Plate_State))
    INTO
        NAME Police_District
        VALUE PaymentIsOutstanding
    ORDER BY Police_District ASC
""").show(max_rows=100)

┌─────────────────────┬─────────────────┬──────────────────────┐
│ License_Plate_State │ Police_District │ PaymentIsOutstanding │
│       varchar       │     varchar     │        int128        │
├─────────────────────┼─────────────────┼──────────────────────┤
│ KS                  │ 1.0             │                 1911 │
│ OR                  │ 1.0             │                  767 │
│ NF                  │ 1.0             │                    9 │
│ CT                  │ 1.0             │                 1066 │
│ AK                  │ 1.0             │                  317 │
│ NC                  │ 1.0             │                 1907 │
│ CO                  │ 1.0             │                 2340 │
│ VA                  │ 1.0             │                 2119 │
│ TN                  │ 1.0             │                 4264 │
│ GA                  │ 1.0             │                 3640 │
│ NS                  │ 1.0             │                   32 │
│ US                  │ 1

In [ ]:
# Aliases in WHERE/GROUP BY/HAVING clauses
# CANNOT be used in JOIN-ON clause!
con.sql("""
    SELECT
        Police_District AS pd,
        CASE WHEN License_Plate_State = 'IL' THEN 'In-State' ELSE 'Out-of-State' END AS License_Plate_State,
        SUM(PaymentIsOutstanding) AS TicketsOutstanding
    FROM ChicagoParkingTickets
    WHERE
        pd IN (1.0, 2.0, 3.0, 4.0)
    GROUP BY
        Police_District,
        CASE WHEN License_Plate_State = 'IL' THEN 'In-State' ELSE 'Out-of-State' END
    ORDER BY SUM(PaymentIsOutstanding) DESC
""")

┌────────┬─────────────────────┬────────────────────┐
│   pd   │ License_Plate_State │ TicketsOutstanding │
│ double │       varchar       │       int128       │
├────────┼─────────────────────┼────────────────────┤
│    1.0 │ In-State            │            1073606 │
│    2.0 │ In-State            │             477357 │
│    3.0 │ In-State            │             418782 │
│    4.0 │ In-State            │             388914 │
│    1.0 │ Out-of-State        │             227079 │
│    2.0 │ Out-of-State        │              72515 │
│    3.0 │ Out-of-State        │              40902 │
│    4.0 │ Out-of-State        │              37860 │
└────────┴─────────────────────┴────────────────────┘

In [ ]:
# COLUMNS() and lambda expressions
# NOTE: cannot use COLUMNS() in GROUP BY clause
con.sql("""
    SELECT
        Police_District AS pd,
        COLUMNS(col -> col LIKE 'Per%')
    FROM ChicagoParkingTickets
    WHERE
        pd IN (1.0, 2.0, 3.0, 4.0)
    LIMIT 10
""")

┌────────┬───────────────────┬────────────────────┬─────────────────────────┬──────────────────────────────────┐
│   pd   │ Per_capita_income │ Percent_unemployed │ Percent_without_diploma │ Percent_households_below_poverty │
│ double │      double       │       double       │         double          │              double              │
├────────┼───────────────────┼────────────────────┼─────────────────────────┼──────────────────────────────────┤
│    1.0 │           65526.0 │                5.7 │                     3.1 │                             14.7 │
│    1.0 │           65526.0 │                5.7 │                     3.1 │                             14.7 │
│    1.0 │           65526.0 │                5.7 │                     3.1 │                             14.7 │
│    1.0 │           59077.0 │                4.9 │                     7.4 │                             13.8 │
│    1.0 │           65526.0 │                5.7 │                     3.1 │                   

In [69]:
# Scalar function chaining and trailing commas
con.sql("""
    SELECT
        Sector,
        Sector.upper().replace(' ', '_').trim() AS Sector_upper,
    FROM ChicagoParkingTickets
    WHERE
        Police_District IN (1.0, 2.0, 3.0, 4.0)
    LIMIT 10
""")

┌─────────────────┬─────────────────┐
│     Sector      │  Sector_upper   │
│     varchar     │     varchar     │
├─────────────────┼─────────────────┤
│ Loop            │ LOOP            │
│ Loop            │ LOOP            │
│ Loop            │ LOOP            │
│ Near South Side │ NEAR_SOUTH_SIDE │
│ Loop            │ LOOP            │
│ Loop            │ LOOP            │
│ Loop            │ LOOP            │
│ Loop            │ LOOP            │
│ Loop            │ LOOP            │
│ Loop            │ LOOP            │
├─────────────────┴─────────────────┤
│ 10 rows                 2 columns │
└───────────────────────────────────┘

In [70]:
con.close()

In [ ]:
con.table("duckdbdata/ChicagoParkingTickets.parquet").show()
con.sql("DESCRIBE 'duckdbdata/ChicagoParkingTickets.parquet'")
# TODO: show loading data into duckdb table
con.close()

# TODO: Replicate SQL Server schema for Chicago Parking Tickets
# TODO: Try out a variety of queries
# Standard operations

# TODO: functionality not in T-SQL


# COPY
# Friendly SQL
# Special join types

# TODO: Demo querying a Pandas DataFrame
# TODO: Demo querying a Polars DataFrame

# TODO: Demo on deleting data and vacuuming

# TODO: Demo on indexing techniques

# TODO: Come up with specific extensions to install
# TODO: Demo on installing extensions

# TODO: SET, PRAGMA, and secrets

# TODO: Demo Explain plans

ConnectionException: Connection Error: Connection already closed!